# 🧬 LangChain 실습 노트북

이 노트북은 **11:00–12:00 LangChain 실습**해보기 위한 자료입니다.

다루는 내용:
- LangChain 기본 컴포넌트: `PromptTemplate`, `ChatOpenAI`, `Chain`
- LCEL (LangChain Expression Language) 기초
- 간단한 Chain/Memory 맛보기
- LangSmith Trace가 어떻게 찍히는지 확인

> ⚠️ 실행 전 `.env` 파일에 `OPENAI_API_KEY`, `LANGCHAIN_API_KEY`, `LANGCHAIN_TRACING_V2=true`, `LANGCHAIN_PROJECT=gccare-rag-workshop` 가 설정되어 있어야 합니다.

## 1. 환경 설정 확인

강의용 레포의 루트(`gccare-langchain-rag-ai/`)에 있는 `.env` 파일을 로드하고,
필요한 패키지가 잘 import 되는지 먼저 확인합니다.

In [7]:
!pip install -r ../requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
from pathlib import Path
import os

from dotenv import load_dotenv

BASE_DIR = Path.cwd().resolve().parents[0]
env_path = BASE_DIR / '.env'
load_dotenv(env_path)

print('OPENAI_API_KEY set?:', bool(os.getenv('OPENAI_API_KEY')))
print('LANGCHAIN_API_KEY set?:', bool(os.getenv('LANGCHAIN_API_KEY')))
print('LANGCHAIN_TRACING_V2:', os.getenv('LANGCHAIN_TRACING_V2'))
print('LANGCHAIN_PROJECT:', os.getenv('LANGCHAIN_PROJECT'))

OPENAI_API_KEY set?: True
LANGCHAIN_API_KEY set?: True
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: gccare-rag-workshop


## 2. LangChain 기본 컴포넌트 맛보기

가장 기본적인 조합인 **PromptTemplate + ChatOpenAI** 를 사용해 봅니다.

- `PromptTemplate`: 프롬프트를 텍스트 템플릿 + 변수 형태로 관리
- `ChatOpenAI`: OpenAI Chat 모델 래퍼


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 GC케어 개발자 교육을 돕는 친절한 조교야.'),
    ('user', '다음 텍스트를 3줄로 요약해줘.\n\n{text}')
])

llm = ChatOpenAI(model='gpt-4o-mini')

sample_text = '이번 교육에서는 LangChain, LangSmith, RAG, Agent 등을 활용해 '
sample_text += 'LLM 기반 서비스 개발 전체 흐름을 체험합니다.'

messages = prompt.format_messages(text=sample_text)
for m in messages:
    print(m.type, ':', m.content)

res = llm.invoke(messages)
print('\n[모델 응답]\n', res.content)

system : 너는 GC케어 개발자 교육을 돕는 친절한 조교야.
human : 다음 텍스트를 3줄로 요약해줘.

이번 교육에서는 LangChain, LangSmith, RAG, Agent 등을 활용해 LLM 기반 서비스 개발 전체 흐름을 체험합니다.

[모델 응답]
 이번 교육에서는 LLM 기반 서비스 개발을 위한 LangChain, LangSmith, RAG, Agent 등의 툴을 활용합니다. 참가자들은 이 도구들을 사용하여 전체 개발 흐름을 직접 체험하게 됩니다. 이를 통해 실제 서비스 개발에 필요한 기술과 방법을 학습할 수 있습니다.


In [4]:
resp = llm.invoke("오늘 강의 목표를 한 줄로 말해줘.")
print(resp.content)

오늘 강의의 목표는 핵심 개념을 명확히 이해하고, 실생활에 적용할 수 있는 방법을 모색하는 것입니다.


## 3. LCEL (LangChain Expression Language) 기초

이번에는 같은 기능을 **LCEL 파이프(|)** 를 사용해 더 간결하게 표현해 봅니다.

`chain = prompt | llm` 형태로 구성하면, `chain.invoke({})` 로 바로 호출할 수 있습니다.

In [3]:
from langchain_core.runnables import Runnable

chain = prompt | llm

res = chain.invoke({'text': sample_text})
print(res.content)

이번 교육에서는 LLM 기반 서비스 개발을 다룹니다. 주요 도구로는 LangChain, LangSmith, RAG, Agent가 포함됩니다. 참가자들은 이들 도구를 활용하여 전체 개발 흐름을 직접 체험할 수 있습니다.


## 4. 간단한 Chain 구성 – 요약 → 톤 변경

두 단계로 구성된 체인을 만들어 봅니다.
- 1단계: 한 문장으로 요약
- 2단계: 더 친근한 톤으로 다시 표현

> 실제 강의 코드에서는 `03_chain_memory.py` 에서 더 자세히 다룹니다.

In [9]:
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SimpleSequentialChain
from langchain_core.prompts import PromptTemplate


summ_prompt = PromptTemplate.from_template(
    '다음 내용을 한 문장으로 요약해줘:\n\n{text}'
)
style_prompt = PromptTemplate.from_template(
    '다음을 더 친근하고 이해하기 쉬운 톤으로 바꿔줘:\n\n{text}'
)

summ_chain = LLMChain(llm=llm, prompt=summ_prompt)
style_chain = LLMChain(llm=llm, prompt=style_prompt)

overall_chain = SimpleSequentialChain(chains=[summ_chain, style_chain])

result = overall_chain.invoke(sample_text)
print(result)

ModuleNotFoundError: No module named 'langchain.chains'

In [10]:
# 🔁 LLMChain / SimpleSequentialChain 대신 LCEL로 구현

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

summ_prompt = PromptTemplate.from_template(
    "다음 내용을 한 문장으로 요약해줘:\n\n{text}"
)
style_prompt = PromptTemplate.from_template(
    "다음을 더 친근하고 이해하기 쉬운 톤으로 바꿔줘:\n\n{text}"
)

# 1단계: 요약 → string 출력
summ_chain = summ_prompt | llm | StrOutputParser()

# string → {"text": string} 로 변환해서 다음 프롬프트에 넣기
to_style_input = RunnableLambda(lambda s: {"text": s})

# 2단계: 톤 변경 → string 출력
style_chain = style_prompt | llm | StrOutputParser()

# 전체 파이프: 요약 → (형 변환) → 톤 변경
overall_chain = summ_chain | to_style_input | style_chain

text = (
    "LangChain과 LangSmith를 활용하면 LLM 기반 애플리케이션의 구성, "
    "디버깅, 모니터링을 훨씬 더 쉽게 할 수 있습니다."
)

result = overall_chain.invoke({"text": text})  # 또는 overall_chain.invoke(text)도 동작
print(result)


LangChain과 LangSmith를 활용하면 LLM 기반 애플리케이션을 더 쉽게 구성하고, 오류를 찾아내고, 모니터링할 수 있어요! 아주 편리하답니다.


## 5. Memory 맛보기 – 대화형 체인

간단한 **ConversationChain + Memory** 로, 이전 대화를 기억하는 챗봇을 만들어 봅니다.

자세한 내용은 `03_chain_memory.py` 세션에서 더 다룹니다.

In [15]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
chat = ConversationChain(llm=llm, memory=memory, verbose=True)

print('--- 첫 번째 질문 ---')
chat.invoke('오늘 GC케어 LangChain 교육의 목표를 한 문장으로 말해줘.')

print('\n--- 두 번째 질문 (이전 답변을 참고해야 함) ---')
res2 = chat.invoke('방금 말한 내용을 두 줄로 다시 정리해줘.')
print('\n최종 응답:', res2['response'])

ModuleNotFoundError: No module named 'langchain.memory'

In [ ]:
# 최신 LangChain 구조에서 권장되는 "대화 메모리" 패턴
# - ConversationChain 없이 LCEL + RunnableWithMessageHistory 사용
# - history는 세션별로 자동 관리

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# llm 이미 있으시면 재사용하세요. 없으면 주석 해제:
# llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 GC케어 LangChain 교육의 친절한 조교야. 간결하고 명확히 답해줘."),
    MessagesPlaceholder("history"),  # ← 과거 대화를 여기에 주입
    ("human", "{input}")
])

chain = prompt | llm  # LCEL

# 세션별 히스토리 저장소 (실무에선 Redis/DB 등으로 대체)
session_store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

chat_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",   # 프롬프트에서 사람 입력 변수명
    history_messages_key="history" # MessagesPlaceholder 키
)

# === 사용 예시 ===
session_id = "demo-user-1"

print('--- 첫 번째 질문 ---')
res1 = chat_with_memory.invoke(
    {"input": "오늘 GC케어 LangChain 교육의 목표를 한 문장으로 말해줘."},
    config={"configurable": {"session_id": session_id}},
)
print(res1.content)

print('\n--- 두 번째 질문 (이전 답변을 참고해야 함) ---')
res2 = chat_with_memory.invoke(
    {"input": "방금 말한 내용을 두 줄로 다시 정리해줘."},
    config={"configurable": {"session_id": session_id}},
)
print('\n최종 응답:', res2.content)


--- 첫 번째 질문 ---
GC케어 LangChain 교육의 목표는 자연어 처리와 데이터 관리 기술을 활용하여 효과적인 언어 모델 응용 프로그램을 개발하는 능력을 기르는 것입니다.

--- 두 번째 질문 (이전 답변을 참고해야 함) ---

최종 응답: GC케어 LangChain 교육의 목표는 자연어 처리와 데이터 관리 기술을 활용하는 것입니다. 이를 통해 효과적인 언어 모델 응용 프로그램 개발 능력을 기르는 것입니다.


## 6. LangSmith에서 Trace 확인하기

이 노트북에서 실행한 모든 LangChain 호출은, `.env` 에 아래와 같이 설정되어 있다면
자동으로 LangSmith에 Trace로 저장됩니다.

```bash
LANGCHAIN_API_KEY=ls-...
LANGCHAIN_TRACING_V2=true
LANGCHAIN_PROJECT=gccare-rag-workshop
```

브라우저에서 [https://smith.langchain.com](https://smith.langchain.com)에 접속해서
해당 프로젝트를 선택하면, 지금까지 실행한 체인/대화들이 어떻게 기록되었는지
트리 구조로 확인할 수 있습니다.

> 권장 활동: 방금 실행한 LCEL 체인과 ConversationChain이 각각 어떤 구조로
> Trace에 남았는지 비교해 보세요.